In [1]:
from __future__ import annotations
from typing import Tuple
import numpy as np

## Invariance Test
Later, in the vibrational solver: 
1. Build an ANM Hessian $H$ from a geometry.
2. Mass-weight it and construct the projector $Q$ to remove rigid-body motion.
3. Diagonalise the projected matrix to get eigenvalues.

If you take the *same* molecule and just spin it and move it in space (rigid transform), the non-zero vibrational eigenvalues must be identical.


## Pure rotation

A rigid rotation means every atom is rotated about some axis through the COM, without stretching distances.

Consider a unit axis $\hat{\mathbf{e}}_\alpha$ (for example, the $x$-, $y$-, or $z$-axis).  
For a small rotation about that axis, atom $i$ is displaced by the cross product:
$$
\Delta \mathbf{R}_i^{(\alpha)} = \hat{\mathbf{e}}_\alpha \times \mathbf{r}_i,
$$
where $\mathbf{r}_i$ are the COM-centred coordinates.

build three such rotational displacement patterns:
- rotation about $x$: $\hat{\mathbf{e}}_x \times \mathbf{r}_i$  
- rotation about $y$: $\hat{\mathbf{e}}_y \times \mathbf{r}_i$  
- rotation about $z$: $\hat{\mathbf{e}}_z \times \mathbf{r}_i$

Stacking those for all atoms gives $R_x, R_y, R_z$ — the last three columns of the rigid-body basis matrix $S$.


## Rotation matrix with preset angle

In [2]:
def rotation_matrix_from_axis_angle(axis: np.ndarray, theta: float) -> np.ndarray:
    """
    Build a 3x3 rotation matrix from a rotation axis and an angle (right-hand rule).

    Uses Rodrigues' rotation formula.

    Args:
        axis: (3,) rotation axis direction vector (does NOT need to be normalised).
        theta: rotation angle [radians].

    Returns:
        Rmat: (3,3) rotation matrix.
    """
    axis = np.asarray(axis, dtype=float)
    if axis.shape != (3,):
        raise ValueError("axis must be shape (3,)")

    # normalise axis
    norm = np.linalg.norm(axis)
    if norm == 0.0:
        raise ValueError("rotation axis has zero length")
    k = axis / norm  # unit vector

    kx, ky, kz = k
    K = np.array([[0.0, -kz,  ky],
                  [kz,  0.0, -kx],
                  [-ky, kx,  0.0]], dtype=float)

    I = np.eye(3)
    ct = np.cos(theta)
    st = np.sin(theta)

    # Rodrigues formula: R = I cosθ + (1-cosθ) k k^T + sinθ [k]_x
    Rmat = ct * I + (1.0 - ct) * np.outer(k, k) + st * K
    return Rmat

## Rotation matrix with random angle

In [3]:
def random_rotation_matrix(rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Generate a random 3x3 rotation matrix (uniform over SO(3)).

    Args:
        rng: optional np.random.Generator for reproducibility.

    Returns:
        Rmat: (3,3) proper rotation matrix with det=+1.
    """
    if rng is None:
        rng = np.random.default_rng()

    # random unit quaternion (Marsaglia method)
    u1 = rng.random()
    u2 = rng.random() * 2.0 * np.pi
    u3 = rng.random() * 2.0 * np.pi

    q0 = np.sqrt(1.0 - u1) * np.sin(u2)
    q1 = np.sqrt(1.0 - u1) * np.cos(u2)
    q2 = np.sqrt(u1)       * np.sin(u3)
    q3 = np.sqrt(u1)       * np.cos(u3)

    # quaternion -> rotation matrix
    # (x,y,z,w) vs (q0,q1,q2,q3) naming is annoying in literature; here (q0,q1,q2,q3)
    x, y, z, w = q0, q1, q2, q3

    Rmat = np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),         2*(x*z + y*w)],
        [2*(x*y + z*w),         1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [2*(x*z - y*w),         2*(y*z + x*w),         1 - 2*(x*x + y*y)]
    ], dtype=float)

    return Rmat

## Finite rigid transformation on coordinates

### Rotation matrix

describe rotation with a $3 \times 3$ matrix $U$ satisfying:
$$
U^\top U = I, \quad \det(U) = +1.
$$

Such a matrix rotates a vector without changing its length.

To build $U$ from "axis + angle", we can use Rodrigues' rotation formula.  
Given:
- a rotation axis $\mathbf{k}$,
- a rotation angle $\theta$ (radians),

we first normalise
$$
\hat{\mathbf{k}} = \frac{\mathbf{k}}{\|\mathbf{k}\|},
$$

then define the skew-symmetric matrix
$$
K =
\begin{bmatrix}
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0
\end{bmatrix},
$$
where $(k_x,k_y,k_z)$ are the components of $\hat{\mathbf{k}}$.

Rodrigues' formula for the rotation matrix is
$$
U(\hat{\mathbf{k}},\theta) 
= I \cos\theta
+ (1 - \cos\theta)\,\hat{\mathbf{k}}\hat{\mathbf{k}}^\top
+ \sin\theta \, K.
$$

### Rigid transform

A general rigid transform is:
$$
\mathbf{R}_i' = U \, \mathbf{R}_i + \mathbf{t},
$$
where $U$ is a $3 \times 3$ rotation matrix and $\mathbf{t}$ is a $3$-component translation vector. The output is the transformed coordinates $R'$.


In [4]:
def apply_rigid_transform(
    R: np.ndarray,
    rotation: np.ndarray,
    translation: np.ndarray,
) -> np.ndarray:
    """
    Apply a global rigid transformation to coordinates:
        R' = R * rotation^T + translation
    (equivalently translation + rotation acting on each vector)

    Args:
        R: (N,3) coordinates [Å].
        rotation: (3,3) rotation matrix.
        translation: (3,) translation vector [Å].

    Returns:
        R_new: (N,3) transformed coordinates.
    """
    R = np.asarray(R, dtype=float)
    rotation = np.asarray(rotation, dtype=float)
    translation = np.asarray(translation, dtype=float)

    if R.ndim != 2 or R.shape[1] != 3:
        raise ValueError("R must have shape (N,3)")
    if rotation.shape != (3,3):
        raise ValueError("rotation must have shape (3,3)")
    if translation.shape != (3,):
        raise ValueError("translation must have shape (3,)")

    # We can do row-wise dot: R @ rotation.T (so each row is rotated)
    R_rot = R @ rotation.T
    
    return R_rot + translation[None, :]

In [5]:
def random_rigid_transform(
    R: np.ndarray,
    rng: np.random.Generator | None = None,
    translation_scale: float = 1.0,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Create and apply a RANDOM rigid transform (random rotation + random translation).
    
    Args:
        R: (N,3) coordinates [Å].
        rng: np.random.Generator for reproducibility (optional).
        translation_scale: controls how large the random translation is [Å].

    Returns:
        R_new: (N,3) transformed coordinates.
        Rmat: (3,3) rotation actually used.
        tvec: (3,) translation actually used.
    """
    if rng is None:
        rng = np.random.default_rng()

    Rmat = random_rotation_matrix(rng)
    
    # translation drawn from normal distribution scaled by translation_scale
    tvec = rng.normal(loc=0.0, scale=translation_scale, size=(3,))

    R_new = apply_rigid_transform(R, Rmat, tvec)
    return R_new, Rmat, tvec

In [6]:
import numpy as np
from ase.build import molecule

atoms = molecule('H2O')
symbols = atoms.get_chemical_symbols()
R = atoms.get_positions() 

masses = atoms.get_masses() 

com = center_of_mass(R, masses)
Rc, com2 = shift_to_com(R, masses)

print("Original COM:\n", com)
print("Recomputed COM from Rc:\n", center_of_mass(Rc, masses)) # This second COM should be [0,0,0]


NameError: name 'center_of_mass' is not defined

In [ ]:
R_rand, U_rand, t_rand = random_rigid_transform(Rc, translation_scale=1.0)
print("Random rotation matrix:\n", U_rand)
print("Random translation vector:\n", t_rand)
print("atom after random rigid move:\n", R_rand)

Random rotation matrix:
 [[ 0.56203642 -0.7928661  -0.23553856]
 [ 0.0841756   0.33812393 -0.93732955]
 [ 0.82281804  0.50698675  0.25677794]]
Random translation vector:
 [ 0.47760199  1.14171432 -0.75428233]
atom after random rigid move:
 [[ 0.46188427  1.07916538 -0.73714728]
 [-0.0028083   1.89617279 -0.50331422]
 [ 1.20748436  1.38003406 -1.27721834]]


## Testing
Rigid motion must not change internal distances.

For any $i,j$:
$$
|\mathbf{R}_i - \mathbf{R}_j| = |\mathbf{R}_i’ - \mathbf{R}_j’|.
$$

In [ ]:
def pairwise_distances(R):
    N = R.shape[0]
    D = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            D[i,j] = np.linalg.norm(R[i] - R[j])
    return D

Max change in pairwise distances: 1.1102230246251565e-16


In [ ]:
D0 = pairwise_distances(R)
D1 = pairwise_distances(R_rand)

print("Max change in pairwise distances:", np.max(np.abs(D0 - D1))) # This should be ~0 (floating-point ~1e-12)